<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
بسمه تعالی
<p></p>
</font>
<p></p>
<font>
<br>
درس بازیابی پیشرفته اطلاعات
<br>
مدرس: دکتر بیگی
</font>
<p></p>
<br>
<font>
<b>تمرین دوم</b>
</font>
<br>
<br>
موعد تحویل: ... آبان <br>
<br>
<br>
<br>
<font>
دانشگاه صنعتی شریف
<br>
دانشکده مهندسی کامپیوتر
<br>
<br>
</font>
</div>
</font>
</div>

<div dir='rtl'>
<h1> 
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
    <li>مدل‌های برداری</li>
    <li>امتیازدهی و ارزیابی سیستم بازیابی</li>
    <li>مدل‌های احتمالاتی</li>

دیتاست مورد استفاده در این تمرین را می‌توانید در کنار این فایل مشاهده کنید. همچنین لطفا پس از اتمام تمرین یک بار از اول تا آخر نوت‌بوک را اجرا کنید تا مطمئن باشید تمام سل‌ها به درستی کار می‌کنند. 

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید. 
</p>
</div>

In [303]:
import math
import pandas as pd
import numpy as np
from collections import defaultdict
from typing import List, Union
import re
from collections import defaultdict

In [75]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MMowS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MMowS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MMowS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

<div dir='rtl'>
<h1>1.
آماده‌سازی دیتاست
</h1>
<p>
با استفاده از تمرین قبل و عملیات‌هایی که در آنجا برای پیش‌پردازش متون پیاده‌سازی کردید، دیتاست داده شده را لود کنید. قرار است در ادامه با این دیتاست بخش‌های بعدی تمرین را پیاده‌سازی کنید. همچنین در صورتی که اجرای سل‌های بعدی برایتان طول کشید، می‌توانید آن را کوچک کنید. مثلا از ۷۰درصد دیتای آن استفاده کنید. 
</p>
</div>

In [304]:
# load the dataset
data_frame = pd.read_csv('data.csv')
data_frame = data_frame
# print(data_frame.head())
print("primary length:", len(data_frame))

data_frame.fillna("None", inplace=True)
print("length after dropping None columns:", len(data_frame))


primary length: 6185
length after dropping None columns: 6185


In [305]:
# preprocess

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
import re
import string

class Preprocessor:

    def __init__(self):
        # Create a variable of stop words.
        self.stopwords = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()

    def preprocess(self, text):
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        normalized_text = self.normalize(text)
        tokens = self.word_tokenize(normalized_text.lower())
        return tokens

    def normalize(self, text):
        tokens = self.word_tokenize(text.lower())
        filtered_tokens = self.remove_stopwords([token for token in tokens if token not in self.stopwords])
        lemmatizer = WordNetLemmatizer()
        normalized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
        return ' '.join(normalized_tokens)

    def remove_links(self, text):
        url_pattern = re.compile(r'http[s]?://\S+|www\.\S+')
        cleaned_text = re.sub(url_pattern, '', text)
        return cleaned_text

    def remove_punctuations(self, text):
        table = str.maketrans("","")
        return text.translate(str.maketrans('','', string.punctuation))

    def word_tokenize(self, text):
        tokens = word_tokenize(text)
        return tokens

    def remove_stopwords(self, words):
        filtered_tokens = [token for token in words if token not in self.stopwords]
        return filtered_tokens

<div dir='rtl'>
<h2>1-1.
نمایه‌سازی
</h2>
<p>
در این بخش باید برای سامانه یک
Positional Index
بسازید. 
<br>
با توجه به مواردی که در بخش بعد می‌آید و نیاز به جست‌وجو‌ی مجزا و با امتیازدهی متفاوت بر روی بخش‌های مختلف سند مثل عنوان یا چکیده آن، در این قسمت باید نمایه‌ی مناسب برای امکان جست‌وجو‌ در بخش‌های مختلف را پیاده‌سازی کنید.
با استفاده از نمایه‌ی ساخته‌شده باید بتوان شماره‌ی تمامی اسنادی که یک کلمه در آن آمده است و همچنین همه‌ی جایگاه‌های این کلمه در هر بخش از هر سند را پیدا کرد.

برای این بخش می‌توانید از نمایه‌ای که در تمرین اول زدید استفاده کنید. 
</p>
</div>

In [306]:
def construct_positional_indexes(corpus):
    """
    Constructs positional indexes for the processed data.

    Parameters
    ----------
    corpus : DataFrame
        The corpus dataframe containing title, abstract and paperId
        
    Returns
    -------
    dict
        The constructed positional indexes
    """
    def build_keys(token, doc_id, index):
        if token not in index:
            index[token] = {}
        if doc_id not in index[token]:
            index[token][doc_id] = {'title': [], 'abstract': []}

    preprocess = Preprocessor().preprocess
    positional_index = {}
    docs_length = {}
    total_length = {"title": 0, "abstract": 0}

    for index, row in corpus.iterrows():
        doc_id = row['paperId']
        abstract_tokens = preprocess(row['abstract'])
        title_tokens = preprocess(row['title'])

        docs_length[doc_id] = {'title': len(title_tokens), 'abstract': len(abstract_tokens)}
        total_length["title"] += docs_length[doc_id]["title"]
        total_length["abstract"] += docs_length[doc_id]["abstract"]

        for position, token in enumerate(title_tokens):
            build_keys(token, doc_id, positional_index)
            positional_index[token][doc_id]['title'].append(position)


        for position, token in enumerate(abstract_tokens):
            build_keys(token, doc_id, positional_index)
            positional_index[token][doc_id]['abstract'].append(position)
    
    return positional_index, docs_length, total_length
    

corpus = data_frame
positional_index, docs_length, total_length = construct_positional_indexes(corpus)

In [307]:
# print(docs)

<div dir='rtl'>
<h1>2.
مدل‌های برداری و احتمالاتی
</h1>
<p>
در این بخش قصد داریم تا با استفاده از مدل‌های برداری و احتمالاتی، دو سیستم بازیابی اطلاعات طراحی کنیم. در نهایت قرار است سیستم‌های طراحی شده در این بخش را مورد ارزیابی قرار دهیم.

در مدل‌های برداری ما به ازای هر داک که در اختیار داریم و کوئری ورودی یک بردار در فضا در نظر می‌گیریم. در ادامه برای بدست آوردن میزان ارتباط داک‌ها و کوئری، از معیارهایی مانند ضرب داخلی بردارها استفاده می‌کنیم. در این بخش ابتدا بردارهای مربوط به هرکدام را با استفاده از مقادیر 
tf, idf
می‌سازیم و سپس به سراغ محاسبه امتیاز داک‌ها می‌رویم تا در نهایت بتوانیم با استفاده از این امتیازها داک‌هایی مرتبط‌تر را خروجی دهیم و یک سیستم بازیابی کامل را طراحی کنیم. 

در مدل‌های احتمالاتی اساس کار محاسبه 
$P(R | d, q)$
است. در این روش می‌خواهیم به نوعی داک‌ها را به صورت مدل‌های احتمالاتی ببینیم. در درس مشاهده کردید که در انتها مدل‌های احتمالاتی هم رفتاری مشابه با مدل‌های برداری دارند. در ادامه یک سیستم مبتنی بر این مدل‌ها و با روش 
Okapi25
طراحی می‌کنید. 
</p>
</div>

<div dir='rtl'>
<h2>2-1.
ساخت ماتریس TF-IDF
</h2>
<p>
در این مرحله دو تابع
<code dir="ltr">get_tf(token, doc_id)</code>
و
<code dir="ltr">get_idf(token)</code>
را پیاده‌سازی کنید که تابع اول مقدار
tf
توکن ورودی را در شناسه‌ی موردنظر
و تابع دوم مقدار
idf
توکن ورودی را برروی نمایه‌ی کنونی حساب می‌کند و آن را به عنوان خروجی برمی‌گرداند.
</p>
</div>

$$tf_{t, d} = \text{Numbers of } t \text{ in the title of document } d + \text{Numbers of } t \text{ in the abstract of document } d$$
$$idf_t = \log\Biggl(\frac{\text{Number of documents}}{\text{Number of documents that contains } t + 1}\Biggr)$$

In [308]:
def get_tf(token: str, positional_index: dict, doc_id: str) -> int:
    """
    Retrieves the term frequency (TF) of a given token within a specific document.

    Parameters
    ----------
    token : str
        The token for which to retrieve the term frequency.
    positional_index : dict
        A positional index that maps tokens to their positions in documents.
    doc_id : str
        The unique identifier of the document in which to calculate the TF.

    Returns
    -------
    int
        The term frequency (TF) of the given token within the specified document.
    """

    if token in positional_index:
        doc_dict = positional_index[token]
        if doc_id in doc_dict:
            return len(doc_dict[doc_id]['title']) + len(doc_dict[doc_id]['abstract'])
    
    return 0


get_tf('measuring', positional_index, '597c2d96c45e8ad83fc08e5d464d266b68f873ed')

1

In [309]:
def get_idf(token: str, docs: dict) -> float:
    """
    Retrieves the inverse document frequency (IDF) of a given token.

    Parameters
    ----------
    token : str
        The token to retrieve the IDF for.

    Returns
    -------
    float
        The IDF of the given token.
    """

    num_docs = len(docs)
    df = len(docs[token]) if token in docs else 0
    
    if df > 0:
        return math.log((num_docs) / (df + 1))
    else:
        return 0

get_idf('deep', positional_index)

4.45143608604605

<div dir='rtl'>
<p>
در مرحله‌ی نهایی تابع
<code dir="ltr">generate_tfidf_list(corpus, positional_index)</code>
را پیاده‌سازی کنید که با ورودی گرفتن
corpus
و
positional_index
مقادیر
tf-idf
را برای هر
token
به دست می‌آورد و به شکل یک ماتریس به عنوان خروجی برمی‌گرداند.
</p>
</div>

In [310]:
def generate_tfidf_list(corpus, positional_index):
    """
    Generates a list of dictionaries representing documents with associated TF-IDF scores.
    Parameters
    ----------
    corpus : dict
        The corpus containing the processed data.
    positional_index : dict
        The positional index containing the term frequencies and document frequencies.
    Returns
    -------
    list[dict[str, float]]
        The list of dictionaries representing documents with associated TF-IDF scores.
    """

    tfidf = []

    for index, row in corpus.iterrows():

        doc_id = row['paperId']

        doc_tfidf = {}

        for token in positional_index.keys():

            tf = get_tf(token, positional_index, doc_id)
            idf = get_idf(token, positional_index)

            if tf > 0:
                doc_tfidf[token] = tf * idf

        tfidf.append(doc_tfidf)

    return tfidf

tfidf = generate_tfidf_list(corpus, positional_index)

In [311]:
tfidf[:1]

[{'analysis': 3.688221342869387,
  'ground': 7.131651488154816,
  'surface': 5.589107810550745,
  'ultrasonic': 8.712101863715663,
  'face': 6.547138148597665,
  'grinding': 9.048574100336877,
  'silicon': 7.949961811668767,
  'carbide': 9.964864832211031,
  'sic': 9.559399724102867,
  'ceramic': 7.949961811668767,
  'minor': 7.479958182423031,
  'vibration': 8.260116739972606,
  'amplitude': 7.439136187902776,
  'none': 2.932681974853842}]

In [312]:
def show_document_vector(vector, doc_index: int) -> None:
    """
    Prints the non-zero weights and corresponding terms for a document represented as a vector.
    Parameters
    ----------
    vector : list of dict of str:float
        The list of document vectors, where each document vector is a dictionary with term weights.
    doc_id : int
        The document index for which to display the vector.
    Returns
    -------
    None
    """
    vector_list = [(term, weight) for term, weight in vector[doc_index].items() if weight > 0]
    vector_list.sort(key=lambda x: x[1], reverse=True)
    length = math.sqrt(sum(weight ** 2 for _, weight in vector_list))
    normalized = {term: tfidf / length for term, tfidf in vector_list}
    for term, tfidf in vector_list:
        print(f'{term.ljust(16)}\t{tfidf}\t(normalized: {normalized[term]})')

show_document_vector(tfidf, 0)

carbide         	9.964864832211031	(normalized: 0.3519705662613583)
sic             	9.559399724102867	(normalized: 0.3376490690706746)
grinding        	9.048574100336877	(normalized: 0.31960611644812165)
ultrasonic      	8.712101863715663	(normalized: 0.30772152737953945)
vibration       	8.260116739972606	(normalized: 0.29175688936143845)
silicon         	7.949961811668767	(normalized: 0.2808018581009059)
ceramic         	7.949961811668767	(normalized: 0.2808018581009059)
minor           	7.479958182423031	(normalized: 0.26420078560107857)
amplitude       	7.439136187902776	(normalized: 0.262758905478353)
ground          	7.131651488154816	(normalized: 0.25189818978282663)
face            	6.547138148597665	(normalized: 0.2312525017002117)
surface         	5.589107810550745	(normalized: 0.1974137606579896)
analysis        	3.688221342869387	(normalized: 0.13027224918804337)
none            	2.932681974853842	(normalized: 0.1035857237137008)


<div dir='rtl'>
<h2>2-2.
امتیازدهی به سندها
</h2>
<p>
در این بخش می‌خواهیم تا با سه شیوه‌ی متفاوت، داکیومنت‌ها را امتیازدهی کرده و kتا با بالاترین امتیاز را برگردانیم.
روش‌ها به شکل زیر است:

- **ltc.lnc:** در این روش از cosine similarity استفاده می‌کنیم و داده‌ها را بر اساس آن مرتّب می‌کنیم. برای متوجّه شدن نحوه‌ی عملکرد این سیستم، می‌توانید به اسلایدهای درس مراجعه کنید. همچنین دقت کنید که لازم است این امتیازدهی را هم به روش 
term-at-a-time
و هم به روش 
doc-at-a-time
محاسبه کنید.

- **Okapi25:** برای پیاده‌سازی این روش از تساوی زیر استفاده کنید:
</p>
</div>
<div>
<p>

$$ RSV_d = \sum_{t \in q} idf(t)\times\frac{(k_1 + 1)tf(t, d)}{k_1((1 - b) + b\frac{dl(d)}{avg(dl)}) + tf(t, d)} $$
</p>
</div>
<div dir='rtl'>
<p>
برای محاسبه این دو معیار، از توابعی که در بخش قبل پیاده‌سازی کردید استفاده کنید.

در تابع search که مربوط به جست و جوی پرسمان کلی است، به عنوان ورودی پارامتر پرسمان (query)، روش محاسبه امتیاز (method)، تعداد اسنادی که باید برگردانده شود (max-result-count) را ورودی می گیرید. ورودی وزن 
(weight)
نشان می‌دهد که امتیاز نهایی چه وزنی از امتیاز عنوان و چکیده دارد. به زبان دیگر:
</p>
</div>
<div>
<p>

$$ final\_score = weight \times title\_score + (1 - weight) \times abstract\_score $$
</p>
</div>
<div dir='rtl'>
<p>

**توجّه کنید** که تابع نوشته شده، صرفاً یک prototype است و تا وقتی که نیازمندی‌های گفته شده را پیاده کنید، تغییر تابع مشکلی ندارد.
</p>
</div>

In [313]:
def get_tf_scoring(token: str, positional_index: dict, doc_id: str) -> int:

    if token in positional_index:
        doc_dict = positional_index[token]
        if doc_id in doc_dict:
            return len(doc_dict[doc_id]['title']), len(doc_dict[doc_id]['abstract'])
    
    return 0, 0


def get_idf_scoring(token: str, positional_index: dict) -> float:

    num_docs = len(positional_index)
    df_abstract = 0
    df_title = 0
    if token in positional_index:
        for doc_id in positional_index[token].keys():
            df_abstract += 1 if positional_index[token][doc_id]['abstract'] else 0
            df_title += 1 if positional_index[token][doc_id]['title'] else 0
    
    if df_title > 0 or df_abstract > 0:
        return math.log((num_docs) / (df_title + 1)), math.log((num_docs) / (df_abstract + 1))
    else:
        return 0, 0


docs_length
total_length

def get_doc_length(doc_id):
    return docs_length[doc_id]['title'], docs_length[doc_id]['abstract']

In [345]:
import math

def okapi(query_tokens, weight=0.5):
    k1 = 2
    b = 0.75
    
    docs_score = {}
    
    for token in query_tokens:
        if token in positional_index:
            docs = positional_index[token]
            idf_title, idf_abstract = get_idf_scoring(token, positional_index)
            for doc_id in docs.keys():
                tf_title, tf_abstract = get_tf_scoring(token, positional_index, doc_id)
                title_length, abstract_length = get_doc_length(doc_id)
                title_score = (idf_title * (k1 + 1) * tf_title / (k1 * ((1 - b) + b * title_length / total_length["title"]) + tf_title))
                abstract_score = (idf_abstract * (k1 + 1) * tf_abstract / (k1 * ((1 - b) + b * abstract_length / total_length["abstract"]) + tf_abstract))
                if doc_id in docs_score:
                    docs_score[doc_id] += weight * title_score + (1 - weight) * abstract_score
                else:
                    docs_score[doc_id] = weight * title_score + (1 - weight) * abstract_score
                    
    return docs_score
    
def ltc_lnc_tat(query_tokens, weight=0.5):
    docs_score = {}

    for token in query_tokens:
        if token in positional_index:
            for doc_id in positional_index[token].keys():
                title_length, abstract_length = get_doc_length(doc_id)
                title_tf_norm = math.sqrt(title_length)
                abstract_tf_norm = math.sqrt(abstract_length)
                
                idf_title, idf_abstract = get_idf_scoring(token, positional_index)
                tf_title, tf_abstract = get_tf_scoring(token, positional_index, doc_id)

                title_component = (1 + math.log(tf_title)) / title_tf_norm if tf_title > 0 else 0
                abstract_component = (1 + math.log(tf_abstract)) / abstract_tf_norm if tf_abstract > 0 else 0

                doc_score = weight * idf_title * title_component + (1 - weight) * idf_abstract * abstract_component
                if doc_id in docs_score: 
                    docs_score[doc_id] += doc_score
                else:
                    docs_score[doc_id] = doc_score
    
#     print(docs_score.values())
    return docs_score

def ltc_lnc_dat(query_tokens, weight=0.5):
    docs_score = {}

    for index, doc_id in data_frame['paperId'].iteritems():
        title_length, abstract_length = get_doc_length(doc_id)
        title_tf_norm = math.sqrt(title_length)
        abstract_tf_norm = math.sqrt(abstract_length)
        docs_score[doc_id] = 0
        for token in query_tokens:
            if token in positional_index:
                idf_title, idf_abstract = get_idf_scoring(token, positional_index)
                tf_title, tf_abstract = get_tf_scoring(token, positional_index, doc_id)

                title_component = (1 + math.log(tf_title)) / title_tf_norm if tf_title > 0 else 0
                abstract_component = (1 + math.log(tf_abstract)) / abstract_tf_norm if tf_abstract > 0 else 0
#                 print(abstract_component, idf_title)
                docs_score[doc_id] += weight * idf_title * title_component + (1 - weight) * idf_abstract * abstract_component
#     print(docs_score.values())
    return docs_score

def search(query, max_result_count, method='ltc-lnc-tat', weight=0.5):
    """
    Finds relevant documents to query
    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs
    method: 'ltc-lnc-tat' or 'ltc-lnc-dat' or 'okapi25'
    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    preprocess = Preprocessor().preprocess
    query_tokens = preprocess(query)
    
    if method == "okapi25":
        docs_score = okapi(query_tokens, weight)
    elif method == "ltc-lnc-tat":
        docs_score = ltc_lnc_tat(query_tokens, weight)
    elif method == "ltc-lnc-dat":
        docs_score = ltc_lnc_dat(query_tokens, weight)
    
    if max_result_count == -1:
        return docs_score.keys()
    else:
        sorted_keys = sorted(docs_score, key=lambda x: docs_score[x])
#         print(sorted_keys)
        return sorted_keys[::-1][:max_result_count]

<div dir='rtl'>
<h1>3.
مدل‌های زبانی
</h1>
<p>
در این بخش یک سیستم مبتنی بر مدل‌های زبانی پیاده‌سازی می‌کنید. مدل زبانی‌ای که باید استفاده کنید مدل 
unigram
است که می‌توان گفت ساده‌ترین مدل ممکن است. در این مدل ما برای هر داک یک مدل دظر نظر می‌گیریم و احتمال حضور کلمات در مدل یک داک 
نسبت به هم مستقل هستند. در پیاده‌سازی این بخش دستتان خیلی باز است. مدل‌هایی که تشکیل می‌دهید و نحوه 
smoothing 
و همچنین پارامترهای مدل همگی به خودتان برمی‌گردد. 
</p>
</div>

In [346]:
def get_smoothing_prob(token):
    title_length = 0
    abstract_length = 0
    if token in positional_index:
        doc_ids = positional_index[token].keys()
        for doc_id in doc_ids:
            a, b = get_tf_scoring(token, positional_index, doc_id)
            title_length += a
            abstract_length += b
    return (title_length + abstract_length) / (total_length['title'] + total_length['abstract']) 


def lm_search(query, max_result_count, smooth_factor=0.5):
    """
    Finds relevant documents to query
    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                      notice that if max_result_count = -1, then you have to return all docs
    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    preprocess = Preprocessor().preprocess
    query_tokens = preprocess(query)
    docs_score = {}

    smoothing_factor = smooth_factor
    for token in query_tokens:
        if token in positional_index:
            doc_ids = positional_index[token].keys()
            smoothing_prob = get_smoothing_prob(token)
            for doc_id in doc_ids:
                tf_title, tf_abstract = get_tf_scoring(token, positional_index, doc_id)
                title_length, abstract_length = get_doc_length(doc_id)
                doc_prob = (tf_title + tf_abstract) / (abstract_length + title_length)
                if doc_id in docs_score:
                    docs_score[doc_id] += math.log(smoothing_factor * doc_prob + (1 - smoothing_factor) * smoothing_prob)
                else:
                    docs_score[doc_id] = math.log(smoothing_factor * doc_prob + (1 - smoothing_factor) * smoothing_prob)
    
    if max_result_count == -1:
        return docs_score.keys()
    else:
        sorted_keys = sorted(docs_score, key=lambda x: docs_score[x])
        return sorted_keys[:max_result_count]

    
corpus = data_frame
docs = lm_search("deep learning", 10)

In [347]:
docs

['028b15f4e4e6e12ab48385cd274cecaf887e80f5',
 'bd25bbec1c321ab28846b04ee6d269e10e18f54b',
 '84599e152d96b9c33cd216339e8d2a600111a660',
 '625c3decc31a2c3d7ca9f5f84450f701798c28ac',
 '41eaa127a82fe94fc31d06c6f8dfeaa5e0d4cb9d',
 '55ca7a2d71f7b82313b3a67d1713caad92a83ecb',
 '9fd8e5cb7958c1b005096e95ba15d0d92cdfbbaa',
 '7a0324f4a44047260ab3ebbd87a3f4744f1ea945',
 'cc2defd5616b98eda74cb908d66498692753f164',
 'd939289250e4c635ae2d3e80834862d392473100']

<div dir='rtl'>
<h1>4.
ارزیابی عملکرد سامانه
</h1>
<p>
در این بخش معیارهای زیر را برای سیستم‌های طراحی شده پیاده‌سازی کنید. سپس در بخش آخر با فراخوانی توابع پیاده‌سازی شده و ورودی دادن مقادیر موجود در فایل
validation.json
به عنوان ورودی 
actual
توابع، می‌توانید معیارها را برای سیستم‌های بازیابی خودتان بدست بیاورید. 
</p>
</div>

In [348]:
class Evaluation:
    def __init__(self, actual, predicts):
        self.actual = actual
        self.predicts = predicts

    def evaluate(self, require_scores):
        for score_type in require_scores:
            if score_type == 'precision':
                print(f'Precision: {self.precision()}')
            elif score_type == 'recall':
                print(f'Recall: {self.recall()}')
            elif score_type == 'f1':
                print(f'F1 Score: {self.f1()}')
            elif score_type == 'map':
                print(f'Mean Average Precision: {self.map()}')
            elif score_type == 'ndcg':
                print(f'NDCG: {self.ndcg()}')
            elif score_type == 'mrr':
                print(f'Mean Reciprocal Rank: {self.mrr()}')
            else:
                print(f'Unknown evaluation metric: {score_type}')
        return self.recall()

    def precision(self):
        precision_sum = 0
        for actual, predicted in zip(self.actual, self.predicts):
            common_elements = set(actual) & set(predicted)
            precision_sum += len(common_elements) / len(predicted) if len(predicted) > 0 else 0
        precision = precision_sum / len(self.actual) if len(self.actual) > 0 else 0
        return precision

    def recall(self):
        recall_sum = 0
        for actual, predicted in zip(self.actual, self.predicts):
            common_elements = set(actual) & set(predicted)
            recall_sum += len(common_elements) / len(actual) if len(actual) > 0 else 0
        recall = recall_sum / len(self.actual) if len(self.actual) > 0 else 0
        return recall

    def f1(self):
        precision_val = self.precision()
        recall_val = self.recall()
        f1_score = (2 * precision_val * recall_val) / (precision_val + recall_val) if (precision_val + recall_val) > 0 else 0
        return f1_score

    def map(self):
        map_sum = 0

        for actual, predicted in zip(self.actual, self.predicts):
            precision_sum = 0
            relevant_count = 0

            for i, item in enumerate(predicted):
                if item in actual:
                    relevant_count += 1
                    precision_sum += relevant_count / (i + 1)

            average_precision = precision_sum / len(actual) if len(actual) > 0 else 0
            map_sum += average_precision

        map_score = map_sum / len(self.actual) if len(self.actual) > 0 else 0
        return map_score

    def dcg(self, ranking):
        dcg_score = ranking[0]
        for i in range(1, len(ranking)):
            dcg_score += ranking[i] / math.log2(i + 1)
        return dcg_score

    def ndcg(self, k=10):
        ndcg_sum = 0

        for actual, predicted in zip(self.actual, self.predicts):
            sorted_ranking = [1 if item in actual else 0 for item in predicted][:k]

            dcg = self.dcg(sorted_ranking)
            ideal_ranking = sorted([1] * len(actual), reverse=True)[:k]
            idcg = self.dcg(ideal_ranking)

            ndcg_sum += dcg / idcg if idcg > 0 else 0

        ndcg_score = ndcg_sum / len(self.actual) if len(self.actual) > 0 else 0
        return ndcg_score

    def mrr(self):
        mrr_sum = 0

        for actual, predicted in zip(self.actual, self.predicts):
            rank = next((i + 1 for i, item in enumerate(predicted) if item in actual), 0)

            mrr_sum += 1 / rank if rank > 0 else 0

        mrr_score = mrr_sum / len(self.actual) if len(self.actual) > 0 else 0
        return mrr_score


In [349]:
import json
with open('validation.json', 'r') as file:
    data = json.load(file)

actuals = []
queries = []

for query, doc_ids in data.items():
    queries.append(query)
    actuals.append(doc_ids)

require_scores = ["precision", "recall", "f1", "map", "ndcg", "mrr"]

In [350]:
predicts = [search(query, 10, method='ltc-lnc-tat', weight=0.76) for query in queries]

eval_vsm = Evaluation(actuals, predicts)
recall = eval_vsm.evaluate(require_scores)

Precision: 0.6833333333333332
Recall: 0.6833333333333332
F1 Score: 0.6833333333333332
Mean Average Precision: 0.6196097883597883
NDCG: 0.7380567758171104
Mean Reciprocal Rank: 0.9166666666666666


In [352]:
predicts = [search(query, 10, method='ltc-lnc-dat', weight=0.76) for query in queries]

eval_vsm = Evaluation(actuals, predicts)
recall = eval_vsm.evaluate(require_scores)

Precision: 0.6833333333333332
Recall: 0.6833333333333332
F1 Score: 0.6833333333333332
Mean Average Precision: 0.6196097883597883
NDCG: 0.7380567758171104
Mean Reciprocal Rank: 0.9166666666666666


In [353]:
predicts = [search(query, 10, method='okapi25', weight=0.85) for query in queries]
eval_bm = Evaluation(actuals, predicts)
recall = eval_bm.evaluate(require_scores)

Precision: 0.6999999999999998
Recall: 0.6999999999999998
F1 Score: 0.6999999999999998
Mean Average Precision: 0.5825661375661375
NDCG: 0.6809957380766001
Mean Reciprocal Rank: 0.8611111111111112


In [355]:
predicts = [lm_search(query, 10, 0) for query in queries]
eval_lm = Evaluation(actuals, predicts)
recall = eval_lm.evaluate(require_scores)

Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1 Score: 0.6666666666666666
Mean Average Precision: 0.6023677248677249
NDCG: 0.6713456774521499
Mean Reciprocal Rank: 0.7222222222222222
